In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [4]:
PATH='data/'



In [9]:
#data = pd.read_csv(PATH +'yelp_review.csv',nrows = 100000)

In [3]:
#data.head()

In [6]:
#data_business = pd.read_csv(PATH+'yelp_business.csv')

ids =[]


for i in range(0,174567):
    for x in data_business['categories'][i].split(';'):
        if x == 'Restaurants':
            ids.append(i)

In [8]:
#data_ids = data_business[data_business.index.map(lambda x:x in ids)]

In [11]:
#data_all = pd.merge(data,data_ids,on=['business_id'])

In [13]:
#data_train= data_all.loc[0:20000,:]

In [62]:
#data_train.to_csv(PATH+'data_train.csv')

In [15]:
#data_validation = data_all.loc[20000:25000,:]

In [63]:
#data_validation.to_csv(PATH+'data_validation')

In [64]:
#data_test.to_csv(PATH+'data_test')

In [5]:
data_train = pd.read_csv(PATH + 'data_train.csv')

In [6]:
data_train['stars_x'] = data_train['stars_x'].replace(5,'five')
data_train['stars_x'] = data_train['stars_x'].replace(4,'four')
data_train['stars_x'] = data_train['stars_x'].replace(3,'three')
data_train['stars_x'] = data_train['stars_x'].replace(2,'two')
data_train['stars_x'] = data_train['stars_x'].replace(1,'one')

In [7]:
data_validation = pd.read_csv(PATH + 'data_validation')

In [8]:
data_validation['stars_x'] = data_validation['stars_x'].replace(5,'five')
data_validation['stars_x'] = data_validation['stars_x'].replace(4,'four')
data_validation['stars_x'] = data_validation['stars_x'].replace(3,'three')
data_validation['stars_x'] = data_validation['stars_x'].replace(2,'two')
data_validation['stars_x'] = data_validation['stars_x'].replace(1,'one')

In [9]:
data_test = pd.read_csv(PATH + 'data_test')

In [10]:
#data_test = data_all.loc[25000:30000,:]

In [11]:
' '.join(spacy_tok(data_train['text'][0]))

"Super simple place but amazing nonetheless . It 's been around since the 30 's and they still serve the same thing they started with : a bologna and salami sandwich with mustard . \n\n Staff was very helpful and friendly ."

In [12]:
from torchtext import vocab, data

In [13]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [34]:
#FILES = dict(train_df=data_train, val=data_validation)
md = LanguageModelData.from_dataframes(PATH,TEXT,col='text',train_df=data_train,val_df=data_validation,bs=34,bptt=35,test_df=data_test)

In [35]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [36]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(2352, 39832, 1, 2800507)

In [14]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '.', 'the', ',', 'and', 'i', 'a', 'to', 'was', 'it', 'of']

In [15]:
TEXT.vocab.stoi['the']

3

In [16]:
md.trn_ds[0].text[:12]

['super',
 'simple',
 'place',
 'but',
 'amazing',
 'nonetheless',
 '.',
 'it',
 "'s",
 'been',
 'around',
 'since']

In [17]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
  235
  644
   37
   19
  129
 2753
    2
   10
   33
   85
  179
  203
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

In [18]:
next(iter(md.trn_dl))

(Variable containing:
    235      5   2510  ...       5     13   4500
    644      9     51  ...     155    137      4
     37    649     23  ...      40     38   1085
         ...            ⋱           ...         
      2      2  21274  ...      16     12      3
     27      6     22  ...    5937    111   1624
     83     67    407  ...       2    184      4
 [torch.cuda.LongTensor of size 81x64 (GPU 0)], Variable containing:
    644
      9
     51
   ⋮   
     43
     54
      6
 [torch.cuda.LongTensor of size 5184 (GPU 0)])

Train

In [37]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [38]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [39]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [40]:
learner.fit(3e-3, 6, wds=1e-6, cycle_len=1, cycle_mult=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

  7%|▋         | 157/2352 [00:07<01:50, 19.83it/s, loss=8.04]

Exception in thread Thread-20:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



epoch      trn_loss   val_loss                                
    0      4.96862    4.894167  
    1      4.588285   4.485375                                
    2      4.463918   4.334871                                
    3      4.366483   4.259614                                
    4      4.327633   4.213943                                
    5      4.298609   4.185501                                



[4.185501]

In [41]:
learner.save_encoder('adam1_enc')


In [42]:
learner.load_encoder('adam1_enc')

In [26]:
#learner.load_cycle('adam1_enc',2)

In [22]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=4)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      4.310741   4.229945  
    1      4.166206   4.163651                              
    2      4.063045   4.124226                              
    3      4.029001   4.119816                              



[4.119816]

In [23]:
learner.save_encoder('adam3_20_enc')

In [24]:
learner.load_encoder('adam3_20_enc')

In [25]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [26]:

TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))


Test

In [43]:
bs=34
bptt=35

In [61]:
data_train['stars_x'] = data_train['stars_x'].replace('five','four')
data_train['stars_x'] = data_train['stars_x'].replace('two','one')

In [62]:
data_validation['stars_x'] = data_validation['stars_x'].replace('five','four')
data_validation['stars_x'] = data_validation['stars_x'].replace('two','one')

In [30]:
data_train['stars_x'] = str(data_train['stars_x'])

In [31]:
data_validation['stars_x'] = str(data_validation['stars_x'])

In [62]:
class DataFrameDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, text_col, label_col, dfs, **kwargs):
        fields = [("text", text_field), ("label", label_field)]
        examples = []
                
        for key, df in dfs.items():
            for i, row in df.iterrows():
                text = row[text_col]
                label = row[label_col]
                examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, path, col, label, train, valid=None, test=None, **kwargs):
        dfs = {'train': train}

        if valid is not None:
            dfs['valid'] = valid
            has_validation = 'valid'
        else:
            has_validation = None
        if test is not None:
            dfs['test'] = test
            has_test = 'test'
        else:
            has_test = None

        return super().splits(path, text_field=text_field, label_field=label_field, text_col=col, label_col=label, train='train', validation=has_validation, test=has_test,  dfs=dfs, **kwargs)



In [33]:
toxicity_label = data.Field(sequential=False)

splits = DataFrameDataset.splits(
    text_field=TEXT, 
    label_field=toxicity_label,
    path=PATH,
    col="text",
    label="stars_x",
    train=data_train, 
    valid=data_validation)

In [63]:
class DataFrameDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, text_col, label_col, dfs, **kwargs):
        fields = [("text", text_field), ("label", label_field)]
        examples = []
                
        for key, df in dfs.items():
            for i, row in df.iterrows():
                text = row[text_col]
                label = row[label_col]
                examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, path, col, label, train, valid=None, test=None, **kwargs):
        dfs = {'train': train}

        if valid is not None:
            dfs['valid'] = valid
            has_validation = 'valid'
        else:
            has_validation = None
        if test is not None:
            dfs['test'] = test
            has_test = 'test'
        else:
            has_test = None
        return super().splits(path, text_field=text_field, label_field=label_field, text_col=col, label_col=label, train='train', validation=has_validation, test=has_test,  dfs=dfs, **kwargs)



toxicity_label = data.Field(sequential=False)

splits = DataFrameDataset.splits(
    text_field=TEXT, 
    label_field=toxicity_label,
    path=PATH,
    col="text",
    label="stars_x",
    train=data_train, 
    valid=data_validation)



splits[0].examples[16]
t = splits[0].examples[16]
t.label, ' '.join(t.text[:16])

('four',
 'cute place for brunch or a treat from the bakery -- \n\n you get into line')

In [64]:
md2=TextData.from_splits(PATH, splits, bs)

In [65]:
m3 = md2.get_model(opt_fn, 1500, 35, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'adam1_enc')


In [66]:
m3.clip=25.
lrs=np.array([1e-4,1e-3,1e-2])

In [67]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.47908    1.446757   0.234654  



HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.973902   0.858751   0.77527   



[0.8587512, 0.775270107084391]

In [68]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='yelp')

HBox(children=(IntProgress(value=0, description='Epoch', max=14), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.921324   0.746936   0.795918  
    1      0.884535   0.810112   0.788435                    
    2      0.868597   0.80927    0.797439                    
    3      0.811755   0.656399   0.826291                    
    4      0.818372   0.655456   0.829452                    
    5      0.773049   0.627471   0.835134                    
    6      0.806696   0.645019   0.831733                    
    7      0.799256   0.655384   0.827731                    
    9      0.739219   0.616775   0.840816                    
    10     0.815399   0.686596   0.829012                    
    11     0.777513   0.629518   0.834614                    
    12     0.785832   0.620578   0.838495                    
    13     0.75551    0.633077   0.838615                    



[0.6330766, 0.8386154405924738]

In [69]:
m3.load_cycle('yelp', 4)

In [70]:
accuracy_np(*m3.predict_with_targs())

0.8380952380952381